## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv('../Weather_Database/WeatherPy_database.csv')
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Santa Cruz Del Norte,CU,23.1539,-81.9236,68.72,84,36,12.35,scattered clouds
1,1,Chuy,UY,-33.6971,-53.4616,62.53,92,85,5.61,overcast clouds
2,2,Grand Gaube,MU,-20.0064,57.6608,77.40,80,33,1.99,scattered clouds
3,3,Ushuaia,AR,-54.8000,-68.3000,44.26,65,40,25.32,scattered clouds
4,4,Sao Filipe,CV,14.8961,-24.4956,74.95,74,40,10.31,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Santa Cruz Del Norte,CU,23.1539,-81.9236,68.72,84,36,12.35,scattered clouds
1,1,Chuy,UY,-33.6971,-53.4616,62.53,92,85,5.61,overcast clouds
2,2,Grand Gaube,MU,-20.0064,57.6608,77.40,80,33,1.99,scattered clouds
4,4,Sao Filipe,CV,14.8961,-24.4956,74.95,74,40,10.31,scattered clouds
7,7,Muriwai Beach,NZ,-36.8167,174.4500,77.95,68,97,4.09,moderate rain
10,10,Mar Del Plata,AR,-38.0023,-57.5575,63.88,83,0,14.00,clear sky
13,13,Wattegama,LK,6.7989,81.4808,76.12,82,7,2.30,clear sky
15,15,Atuona,PF,-9.8000,-139.0333,78.89,73,4,17.74,clear sky
16,16,Ngunguru,NZ,-35.6167,174.5000,70.66,80,31,12.86,scattered clouds
17,17,Ambatondrazaka,MG,-17.8333,48.4167,65.57,86,26,3.65,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                266
City                   266
Country                263
Lat                    266
Lng                    266
Max Temp               266
Humidity               266
Cloudiness             266
Wind Speed             266
Weather Description    266
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna(how='any')
preferred_cities_df.count()

City_ID                266
City                   266
Country                263
Lat                    266
Lng                    266
Max Temp               266
Humidity               266
Cloudiness             266
Wind Speed             266
Weather Description    266
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Santa Cruz Del Norte,CU,68.72,scattered clouds,23.1539,-81.9236,
1,Chuy,UY,62.53,overcast clouds,-33.6971,-53.4616,
2,Grand Gaube,MU,77.40,scattered clouds,-20.0064,57.6608,
4,Sao Filipe,CV,74.95,scattered clouds,14.8961,-24.4956,
7,Muriwai Beach,NZ,77.95,moderate rain,-36.8167,174.4500,
10,Mar Del Plata,AR,63.88,clear sky,-38.0023,-57.5575,
13,Wattegama,LK,76.12,clear sky,6.7989,81.4808,
15,Atuona,PF,78.89,clear sky,-9.8000,-139.0333,
16,Ngunguru,NZ,70.66,scattered clouds,-35.6167,174.5000,
17,Ambatondrazaka,MG,65.57,scattered clouds,-17.8333,48.4167,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
        lat = row["Lat"]
        lng = row["Lng"]
        params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
        hotels = requests.get(base_url, params=params).json()

   # Grab the first hotel from the results and store the name.
        try:
            hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        except (KeyError, IndexError):
            print("Hotel not found... skipping.")

        hotel_df

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Santa Cruz Del Norte,CU,68.72,scattered clouds,23.1539,-81.9236,Felipon Pon Pon
1,Chuy,UY,62.53,overcast clouds,-33.6971,-53.4616,Nuevo Hotel Plaza
2,Grand Gaube,MU,77.40,scattered clouds,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa
4,Sao Filipe,CV,74.95,scattered clouds,14.8961,-24.4956,Hotel Xaguate
7,Muriwai Beach,NZ,77.95,moderate rain,-36.8167,174.4500,Coast Retreat Muriwai - Golf and Surf Holiday
...,...,...,...,...,...,...,...
687,Bac Lieu,VN,78.19,broken clouds,9.2850,105.7244,Khách sạn Sài Gòn Bạc Liêu
688,Manta,EC,73.18,broken clouds,-0.9500,-80.7333,Balandra Hotel
690,Xai-Xai,MZ,78.17,overcast clouds,-25.0519,33.6442,Complexo da Familia
691,Umtata,ZA,60.75,broken clouds,-31.5889,28.7844,LLZZ Boutique Lodge


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df=hotel_df.dropna(how='any')
hotel_df

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Santa Cruz Del Norte,CU,68.72,scattered clouds,23.1539,-81.9236,Felipon Pon Pon
1,Chuy,UY,62.53,overcast clouds,-33.6971,-53.4616,Nuevo Hotel Plaza
2,Grand Gaube,MU,77.40,scattered clouds,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa
4,Sao Filipe,CV,74.95,scattered clouds,14.8961,-24.4956,Hotel Xaguate
7,Muriwai Beach,NZ,77.95,moderate rain,-36.8167,174.4500,Coast Retreat Muriwai - Golf and Surf Holiday
...,...,...,...,...,...,...,...
687,Bac Lieu,VN,78.19,broken clouds,9.2850,105.7244,Khách sạn Sài Gòn Bạc Liêu
688,Manta,EC,73.18,broken clouds,-0.9500,-80.7333,Balandra Hotel
690,Xai-Xai,MZ,78.17,overcast clouds,-25.0519,33.6442,Complexo da Familia
691,Umtata,ZA,60.75,broken clouds,-31.5889,28.7844,LLZZ Boutique Lodge


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template ="""
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Weather Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig
# 11b. Display the figure


Figure(layout=FigureLayout(height='420px'))